# StyleGAN

---

Style Transfer에 기반한 새로운 Generator구조인 StyleGAN에 대해 알아보겠습니다. 

# 1. Style Transfer, PGGAN

---

StyleGAN에 들어가기에 앞서 Style Transfer와 PGGAN 대해 잠시 알아보겠습니다.

Style Transfer는 쉽게 설명하면 어떤 이미지1에서는 특성인 Style을 추출하고, 다른 이미지2에서는 Contents를 추출해 이를 합성하는 과정을 말합니다. 예시로 살펴보겠습니다.

예) 종이가방 Style : 종이, Contents: 가방

PGGAN은 기존의 GAN처럼 모든 스케일에 대해 동시에 학습하지 않고, 대략적인 이미지 구성을 파악하고 단계적으로 세부적인 부분에 주목하는 방향으로 진행됩니다. 즉, 점진적으로 낮은 해상도부터 높은 해상도까지 생성해내는 대표적인 생성모델로 StyleGAN의 토대가 되는 모델입니다.

# 2. StyleGAN

---

![Untitled](https://user-images.githubusercontent.com/37393115/186621876-3c440f4e-956d-48fb-a2e2-1ff573aff30d.png)

StyleGAN의 generator의 전체적인 구조입니다.

기존의 Generator는 Input latent vector가 직접 Convolution, upsampling을 거쳐 이미지로 변환되는 구조였었습니다. 그러나 style-baseed generator를 보게되면 Fully connected layer로 구성된 Mapping network를 지나 intermediate Latent Vector로 변환된 후 Synthesis network를 거치면서 변환되는 이미지에 스타일을 입히는 역할을 수행합니다. 이를 통해 다양한 스타일을 이미지에 부여할 수 있게됩니다.

입력층에서 기존과 다른 두가지 특징을 살펴볼 수 있습니다.

1. **Mapping Network가 존재**
2. **normalize에 AdaIN이 사용**

# 3.Mapping Network

---

StyleGAN의 가장 큰 특징은 input vector로 부터 직접 이미지를 생성하지 않고 FC layer로 이루어진 Mapping Network를 거쳐생성된 intermediate vector로 먼저 변환 후 이미지를 생성하는 것입니다.

![Untitled (1)](https://user-images.githubusercontent.com/37393115/186622162-312c1c9a-65ce-4b11-b846-484a962e83f3.png)

![Untitled (2)](https://user-images.githubusercontent.com/37393115/186622182-8b37e9cb-70e6-4db0-a152-747e85320793.png)

기존의 방법에서처럼 input vector로부터 이미지를 직접 생성할 경우, 고정된 input의 분포에 학습 이미지의 분포를 맞춰야 한다는 한계가 있습니다. 

이로 인해 visual attribute가 input space에 비교적 부정확하게 mapping이 될 것입니다. 즉,  input vector로 visual attribute를 조절하기가 매우 어려워집니다.

이를 해결하는데 **Mapping Network**를 사용했습니다! 

이를 사용할 경우 고정된 w의 분포를 따르지 않아도 됩니다. 보다 유동적인 공간에 mapping할 수 있습니다. 따라서 visual attribute를 조절하기가 용이해집니다. 이러한 특징을 **disentanglement**라고 합니다.

- **entangle**
    - 서로 얽혀 있는 상태여서 특징 구분이 어려운 상태
- **disentangle**
    - 각 style들이 잘 구분 되어있는 상태 어떤 위치에 어떤 특성을 가지고 있는지 명확하게 파악이 가능한 상태
    - 선형적으로 변수를 변경했을 때 어떤 결과물의 feature인지 예측할 수 있는 상태

![Untitled (3)](https://user-images.githubusercontent.com/37393115/186622359-28bad911-4df0-4bb6-a1ad-8e6f1d1c83e6.png)

앞서 생성된 w를 활용해 Synthesis Network가 이미지를 생성하는 과정에서 Style을 입히는데 사용합니다. 

Synthesis Network는 이미지를 convolution과 upsampling을 이용해 이미지를 확장시켜갑니다. 이전의 PGGAN과 다른점은 Normalization과정에서 **AdalN**을 사용한다는 점입니다.

# 4. AdaIN(Adaptive Instance Normalization)

---

![Untitled (4)](https://user-images.githubusercontent.com/37393115/186622394-b50b5ed1-671e-4f76-8990-6c666b5b15cb.png)

AdalN의 공식을 살펴보겠습니다.  $y_{s,i}$ 와 $y_{b,i}$ 가 보입니다. 

- 여기서 $x_i$는 feature map이며 이를 normalization합니다. 그리고 scaling과 bias를 적용한  style y를 각각 적용해줍니다.  ( $y_{s,i}$ 와 $y_{b,i}$)
- 먼저 $y_{s,i}$를 곱해주고 상수를 더합니다.
- $y_{s,i}$ 와 $y_{b,i}$는 FC layers의 결과인 w를 Affine Transformation을 거쳐 모양을 맞추고 Style을 입혀줍니다.
    - 아핀변환은 두 아핀공간(Affine space) U, V 간 공선점(Collinear point)을 유지하는 동형사상(Isomorphism)으로 정의됩니다. 동형사상은 전단사(Bijective)인 선형사상*(*일대일대응 함수*)*을 말하며, 아핀공간은 원점이 존재하지 않는 벡터공간을 말합니다.
    
                                                        Y = f(x) = WX+b
    

수식을 살펴보면 표준편차로 나누고 평균으로 뺀 값을 정규화시키는 과정으로 볼 수 있습니다.

AdalN의 역할을 이해하고 다시 전체적인 프로세스를 살펴보겠습니다.

![Untitled (5)](https://user-images.githubusercontent.com/37393115/186622749-1d81d65e-4ee1-4d5b-b15a-3a9a8f478568.png)


FC layer층을 통과해 산출된 W가 AdalN을 통해 style을 입힐 때 Shape이 안맞아 Affine Transformation을 통해 shape을 맞춥니다.

layer를 거치면서 학습의 안정성을 위해 normalization을 각 layer에 추가하는데 StyleGAN에서는 AdalN이 수행합니다.

 $y_{s,i}$ 와 $y_{b,i}$의 연산과정은 Style을 입히는 과정이라고 볼 수 있습니다.

AdalN에서 정규화를 진행할떄 마다 한번 수행에 하나만 w가 더해지기 때문에 하나의 style이 각각의 scale에서만 영향을 끼칠 수 있게 해주는 역할을 합니다. Style을 옳게 분리하는데 AdalN이 효과적이라고 볼 수 있습니다.(어떤 층에서 어떤 Style이 적용되었는지 확인이 용이함)

# 5. Stochastic Variation, Style Mixing

---

## 1. Stochastic Variation

---

사람의 나이와 인종 등 다양한 확률에 따라 머리카락과 수염 주름 등 특성이 다르게 나타나는 경향이 있습니다. 이런 요소들을 명확하게 캐치할 수 있으면 더 사실적인 이미지를 생성할 수 있습니다. StyleGAN에서는 이를 표현하기 위해 Sythesis Network의 각 층마다 random noise를 추가해 줍니다. 확률적 정보를 따로 추가해주면 input latent vector는 이미지의 더 중요한 정보(인종, 성별, 나이 등)을 표현하는데만 집중해 더 좋은 이미지를 생성할 수 있습니다.

## 2. Style Mixing

---

Mapping Network로부터 나온 intermediate vector는 Sythesis Network의 모든 layer에 대한 Style을 표현하도록 학습됩니다. 이 과정에서 여러 layer의 style들이 상관관계가 있게 되는 위험을 발생시킬 수 있습니다. 이를 막기위해 Style Mixing 을 사용합니다. 학습시에 input vector로부터 두개의 w를 만들고 초반에 학습하는 내용과 후반에 학습하는 내용을 다르게 적용해 학습합니다. style이 교체되는 Layer를 랜덤하게 결정해 상관관계가 생기는 현상을 방지합니다.(Dropout과 비슷함) 즉, 스타일을 뚜렷하게 구분해 상관관계가 생기지 않도록 해 스타일 적용이 명확하게 만들어줍니다.

![Untitled (6)](https://user-images.githubusercontent.com/37393115/186623047-191608f2-a5c4-4edd-9359-18c78ddb0791.png)

![Untitled (7)](https://user-images.githubusercontent.com/37393115/186623066-cc9f1c19-f461-43e8-b70a-49cf4672154a.png)

# 6. Contribution

---

- Disentanglement(다양한 특징들이 잘 분류되어 있는 것)라는 특성을 향상 시켰고, 고해상도 얼굴 데이터셋을 발표하였을 뿐더러 새로운 성능 지표를 제안했다는 점에서 StyleGAN의 Contribution을 확인할 수 있었습니다.